# Documentation

Dans ce notebook nos présentons et expliquons les différentes classes en fonctions que nous avons implementé pour l'étude technique.

## Reconciliation avec AGROVOC

Pour pouvoir chercher des concepts dans AGROVOC nous utilisons un outil de reconciliation qui nous a ete fourni. Il est tres simple a utiliser mais prend beaucoup de temps pour s'initialiser.

Pour creer une instance d'`Agrovoc` on doit passer un graphe vierge au constructeur. Celui-ci va charger l'ontologie dedans. 

In [7]:
from agrovoc import Agrovoc
from rdflib import Graph

g = Graph()
agrovoc = Agrovoc(g)

Now loading the dictionary...


  0%|          | 15/36938 [00:00<04:14, 144.89it/s]

Now indexing the dictionary...


100%|██████████| 36938/36938 [03:09<00:00, 195.36it/s]


Et maintenant il est possible de faire des requettes dans AGROVOC en utilisant la methode `Agrovoc.find_with_agrovoc()`:

In [9]:
agrovoc.find_with_agrovoc('Cameroun')

{('http://aims.fao.org/aos/agrovoc/c_1229', 'Cameroun')}

## Extraction

Dans cette partie nous allons expliciter les differentes fonctions qui se situent dans `grobid_utils`.


In [0]:
from grobid_utils import *


### `pdf_to_xml(url)`

Cette fonction prend en argument l'`url` d'un pdf d'une publication scientifique et retourne son `xml` TEI sous forme d'une chaine de characteres. 

In [23]:
xml = pdf_to_xml('http://agritrop.cirad.fr/557447/1/document_557447.pdf')
xml[:1000]

'<?xml version="1.0" encoding="UTF-8"?>\n<TEI xml:space="preserve" xmlns="http://www.tei-c.org/ns/1.0" \nxmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" \nxsi:schemaLocation="http://www.tei-c.org/ns/1.0 /home/lopez/grobid/grobid-home/schemas/xsd/Grobid.xsd"\n xmlns:xlink="http://www.w3.org/1999/xlink">\n\t<teiHeader xml:lang="pt">\n\t\t<fileDesc>\n\t\t\t<titleStmt>\n\t\t\t\t<title level="a" type="main">Estimation of the mechanical properties of wood ... ESTIMATION OF THE MECHANICAL PROPERTIES OF WOOD FROM Eucalyptus urophylla USING NEAR INFRARED SPECTROSCOPY ESTIMATIVA DE PROPRIEDADES MECÂNICAS DA MADEIRA DE Eucalyptus urophylla USANDO A ESPECTROSCOPIA NO INFRAVERMELHO PRÓXIMO</title>\n\t\t\t</titleStmt>\n\t\t\t<publicationStmt>\n\t\t\t\t<publisher/>\n\t\t\t\t<availability status="unknown"><licence/></availability>\n\t\t\t</publicationStmt>\n\t\t\t<sourceDesc>\n\t\t\t\t<biblStruct>\n\t\t\t\t\t<analytic>\n\t\t\t\t\t\t<author>\n\t\t\t\t\t\t\t<persName xmlns="http://www.tei-c.org/ns

### `extract_entities(xml, lang='fr', mode='fulltext')`

Cette fonction prend en argument un string de `xml` TEI et rend un string de `json` avec les entites. `lang` peut etre soit `'fr'` soit `'en'` et le mode peut etre soit `'fulltext'` comme par defaut, soit `'abstract'` pour extraire uniquement les entites de l'abstract. 


In [34]:
json_text = extract_entities(xml)
json_text[:1000]

'{"runtime": 3338, "nbest": false, "text": "  INTRODUCTION Properties relating to wood resistance include bend strength  also known as modulus of rupture  compressive strength  tensile strength  shear strength  stiffness  among others  And properties relating to wood elasticity include modulus of elasticity  Poisson coefficient  plasticity and resilience   KOLLMANN   CÔTÉ JUNIOR 1968    Strength in compression parallel to grain is the maximum load in the direction of the length a sample is ANDRADE  C  R  et al  capable of sustaining   KOLLMANN   COTÊ JÚNIOR 1968    Modulus of rupture reflects a sample\'s ability to resist deformation when it is subjected to bending and is equivalent to the maximum moment shown by the sample   GREEN et al  1999     Klock  2000   reports that high MOE values indicate a material\'s high strength and low deformation characteristics  which means it will qualify for structural works  Typically  mechanical characterization of wood is done by destructive analy

### `intersection(set1, set2)`

Cette fonction calcule l'intersection entre deux ensembles de concepts si les concepts sont sous la forme de tuples comme ceux retournes par la methode `Agrovoc.find_with_agrovoc()`

In [28]:
set1 = {('http://aims.fao.org/aos/agrovoc/c_1229', 'Cameroun'),
        ('http://aims.fao.org/aos/agrovoc/c_6970', 'Senegal'),
        ('http://aims.fao.org/aos/agrovoc/c_5993', 'Plante'),
        ('http://aims.fao.org/aos/agrovoc/c_4795', 'Souris')}

set2 = {('http://aims.fao.org/aos/agrovoc/c_1229', 'cameroun'),
        ('http://aims.fao.org/aos/agrovoc/c_6693', 'ruminant'),
        ('http://aims.fao.org/aos/agrovoc/c_5993', 'plante'),
        ('http://aims.fao.org/aos/agrovoc/c_1390', 'chat')}

intersection(set1, set2)

{('http://aims.fao.org/aos/agrovoc/c_1229', 'Cameroun'),
 ('http://aims.fao.org/aos/agrovoc/c_5993', 'Plante')}

### `fetch_entities(json_text)`

Recupere les entites et les categories d'un `json` sous forme de string (qui est sorti de entity-fishing donc de la fonction `extract_entities`) et retourne une liste de strings

In [37]:
concepts = fetch_entities(json_text)
concepts[30:35]

['Eucalyptus grandis and E urophylla',
 'Pinus',
 'HEIN',
 'Larix and concluded',
 'samples']

### `to_agrovoc(concepts, agrovoc)`

Transforme la liste de strings `concepts` en un ensemble de concepts agrovoc. Necessite une instance initialisee de `Agrovoc` car utilise la methode de recherche. 

In [30]:
to_agrovoc(['souris', 'chat', 'balais'], agrovoc)

{('http://aims.fao.org/aos/agrovoc/c_1390', 'chat'),
 ('http://aims.fao.org/aos/agrovoc/c_4795', 'souris')}